<a href="https://colab.research.google.com/github/ljy9969/Study/blob/master/colab(0422)cat_dog_small_fine_tuning(EfficientNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install efficientnet

     |████████████████████████████████| 50 kB 4.2 MB/s 


In [5]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 7.9 MB/s 


In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet
import efficientnet.tfkeras as efn # Pretrained Network
from tensorflow.keras.models import Sequential, Model # Model : Functional API(입/출력만 입력해주면 됨)
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import GlobalAveragePooling2D # AveragePooling
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
import cv2 as cv # opencv(Image Resize)

In [9]:
train_dir = '/content/drive/MyDrive/Colab 멀캠 이지연/cat_dog_small/train'
valid_dir = '/content/drive/MyDrive/Colab 멀캠 이지연/cat_dog_small/validation'

IMAGE_SIZE = 256
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

# ImageDataGenerator를 위한 전처리 함수(Resize)
def generate_preprocessing(img):
  img = cv.resize(img, (IMAGE_SIZE, IMAGE_SIZE)) # 256 * 256
  return img

# ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=generate_preprocessing,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1/255,
                                   preprocessing_function=generate_preprocessing)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=['cats', 'dogs'],
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary') # target size를 잡아줄 필요가 없음. 전처리 함수에 정의되어 있기 때문

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    classes=['cats', 'dogs'],
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary',
                                                    shuffle=False) # 섞어서 가져오지 마라. default는 True

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [16]:
# Pretrained Network

pretrained_network = efn.EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)

pretrained_network.trainable = False # 동결

# print(pretrained_network.summary())
# for idx, layer in enumerate(pretrained_network.layers): # BN layer 찾아서 대체할 수 있음
#   print('{}번 : {}'.format(idx, layer.name)) # stem_bn (BatchNormalization) 확인

model = Sequential()

model.add(pretrained_network)

model.add(GlobalAveragePooling2D())
model.add(Dense(units=1,
                activation='sigmoid'))

# Early Stopping
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   patience=5,
                   verbose=1)

# Checkpoint
model_checkpoint = './{epoch:06d}-{val_acc:0.6f}--{acc:0.6f}.ckpt'

checkpointer = ModelCheckpoint(
    filepath=model_checkpoint,
    verbose=1,
    period=2, # 옵션명이 save_freq로 대체됨
    save_best_weights=True,
    mode='auto',
    monitor='val_acc'
)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['acc']) # Checkpoint의 명칭과 맞춰줘야 출력됨

history = model.fit(train_generator,
                    steps_per_epoch=(2000 // BATCH_SIZE),
                    epochs=30,
                    validation_data=valid_generator,
                    validation_steps=(1000 // BATCH_SIZE),
                    callbacks=[es, checkpointer],
                    verbose=1)

# loss: 0.1931 - acc: 0.9290 - val_loss: 0.0729 - val_acc: 0.9890
# 다음 순서는 Fine Tuning! 학습 시간이 너무 오래 걸리므로 생략함
# Fine Tuning : Pretrained Network의 layer 몇 개의 동결을 풀고 다시 학습(for Classifier)

Epoch 1/30
250/250 [==============================] - 795s 3s/step - loss: 0.5805 - acc: 0.7435 - val_loss: 0.4513 - val_acc: 0.9310
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.4792 - acc: 0.8325
Epoch 2: saving model to ./000002-0.956000--0.832500.ckpt


INFO:tensorflow:Assets written to: ./000002-0.956000--0.832500.ckpt/assets


INFO:tensorflow:Assets written to: ./000002-0.956000--0.832500.ckpt/assets


250/250 [==============================] - 90s 360ms/step - loss: 0.4792 - acc: 0.8325 - val_loss: 0.3526 - val_acc: 0.9560
Epoch 3/30
250/250 [==============================] - 46s 184ms/step - loss: 0.4173 - acc: 0.8590 - val_loss: 0.2901 - val_acc: 0.9630
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.3731 - acc: 0.8730
Epoch 4: saving model to ./000004-0.968000--0.873000.ckpt


INFO:tensorflow:Assets written to: ./000004-0.968000--0.873000.ckpt/assets


...

INFO:tensorflow:Assets written to: ./000028-0.989000--0.930500.ckpt/assets


250/250 [==============================] - 92s 367ms/step - loss: 0.1872 - acc: 0.9305 - val_loss: 0.0758 - val_acc: 0.9890
Epoch 29/30
250/250 [==============================] - 46s 185ms/step - loss: 0.1847 - acc: 0.9300 - val_loss: 0.0739 - val_acc: 0.9890
Epoch 30/30
250/250 [==============================] - ETA: 0s - loss: 0.1931 - acc: 0.9290
Epoch 30: saving model to ./000030-0.989000--0.929000.ckpt


INFO:tensorflow:Assets written to: ./000030-0.989000--0.929000.ckpt/assets


INFO:tensorflow:Assets written to: ./000030-0.989000--0.929000.ckpt/assets


250/250 [==============================] - 92s 370ms/step - loss: 0.1931 - acc: 0.9290 - val_loss: 0.0729 - val_acc: 0.9890


In [ ]:
# Fine Tuning : Pretrained Network의 layer 몇 개의 동결을 풀고 다시 학습(for Classifier)